In [1]:
import torch
import torchvision.models
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import skimage.transform
import skimage.io
from skimage.transform import resize

import warnings; warnings.simplefilter('ignore')
import os
%matplotlib inline
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
# load data
train_X = np.load("train_X.npy")
train_y = np.load("train_y.npy")
valid_X = np.load("valid_X.npy")
valid_y = np.load("valid_y.npy")

# transform for torch tensor
train_X = torch.from_numpy(train_X).type(torch.FloatTensor)
train_y = torch.from_numpy(train_y).type(torch.LongTensor)
valid_X = torch.from_numpy(valid_X).type(torch.FloatTensor)

valid_dataset_path = "./hw3-train-validation/validation/"

# construct id list
valid_image_id_list = sorted(list(set([item.split("_")[0] for item in os.listdir(valid_dataset_path)])))


In [3]:
class fcn32(nn.Module):
    def __init__(self, num_classes, pretrained = True):
        super(fcn32, self).__init__()
        self.vgg = torchvision.models.vgg16(pretrained=True)
        # class torch.nn.ConvTranspose2d
        # (in_channels, out_channels, kernel_size, stride=1, padding=0, 
        # output_padding=0, groups=1, bias=True, dilation=1)
        self.vgg.classifier = nn.Sequential(
            nn.ConvTranspose2d(512, num_classes, 32 , 32 , 0, bias=False)
        )
    def  forward (self, x) :        
        x = self.vgg.features(x)
#         print(x.size())
        x = self.vgg.classifier(x)
        return x

def mean_iou_score(pred, labels):
    '''
    Compute mean IoU score over 6 classes
    '''
    mean_iou = 0
    for i in range(6):
        tp_fp = np.sum(pred == i)
        tp_fn = np.sum(labels == i)
        tp = np.sum((pred == i) * (labels == i))
        iou = tp / (tp_fp + tp_fn - tp)
        mean_iou += iou / 6
        print('class #%d : %1.5f'%(i, iou))
    return mean_iou

In [ ]:
model = fcn32(7).cuda()
model = torch.nn.DataParallel(model).cuda()

optimizer = optim.Adam(model.parameters(),lr=0.0002, betas=(0.9, 0.999))
criterion = nn.NLLLoss2d()

In [ ]:
BATCH_SIZE = 64

# training 
for epoch in range(20):
    print("Epoch:", epoch)
    running_loss = 0.0
    total_length = len(train_X)
    # shuffle
    perm_index = torch.randperm(total_length)
    train_X_sfl = train_X[perm_index]
    train_y_sfl = train_y[perm_index]
    
    # construct training batch
    for index in range(0,total_length ,BATCH_SIZE):
        if index+BATCH_SIZE > total_length:
            break
        # zero the parameter gradients
        optimizer.zero_grad()
        input_X = train_X_sfl[index:index+BATCH_SIZE]
        input_y = train_y_sfl[index:index+BATCH_SIZE]

        # use GPU
        input_X = Variable(input_X.cuda())
        input_y = Variable(input_y.cuda())

        # forward + backward + optimize
        outputs = model(input_X)
        outputs = F.log_softmax(outputs, dim= 1)
        loss = criterion(outputs, input_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        
    print("Loss:",running_loss/(total_length/BATCH_SIZE))
    
    # validation stage
    model.eval()
    pred = torch.FloatTensor()
    pred = pred.cuda()
    for i in range(len(valid_X)):
        input_X_valid = Variable(valid_X[i].view(1,3,256,256).cuda())
        output = model(input_X_valid)
        pred = torch.cat((pred,output.data),0)
    pred = pred.cpu().numpy()
    print("resize...")
    
    pred = np.argmax(pred,1)
    
    pred_512 = np.array([resize(p,output_shape=(512,512), order=0,preserve_range=True,clip=True) for p in pred])
    mean_iou = mean_iou_score(pred_512, valid_y)
    print("mean iou score",mean_iou)
    if epoch+1 in [1,10,20]: # save pred map
        # decoding stage
        n_masks = len(valid_X)
        masks_RGB = np.empty((n_masks, 512, 512, 3))
        for i, p in enumerate(pred_512):
            masks_RGB[i, p == 0] = [0,255,255]
            masks_RGB[i, p == 1] = [255,255,0]
            masks_RGB[i, p == 2] = [255,0,255]
            masks_RGB[i, p == 3] = [0,255,0]
            masks_RGB[i, p == 4] = [0,0,255]
            masks_RGB[i, p == 5] = [255,255,255]
            masks_RGB[i, p == 6] = [0,0,0]
        masks_RGB = masks_RGB.astype(np.uint8)
        # save them
        print("save image")
        output_dir = "./output_folder_"+str(epoch+1)
        for i, mask_RGB in enumerate(masks_RGB):
            skimage.io.imsave(os.path.join(output_dir,valid_image_id_list[i]+"_mask.png"), mask_RGB)
        torch.save(model.state_dict(), "./models/"+ str(mean_iou)[:4]+".pkl")
    print("\n")
    model.train()

Epoch: 0
Loss: 1.0176083984902757
resize...
class #0 : 0.58129
class #1 : 0.79263
class #2 : 0.00003
class #3 : 0.68576
class #4 : 0.59463
class #5 : 0.42442
mean iou score 0.5131255775338782
save image


Epoch: 1
Loss: 0.6267028937234643
resize...
class #0 : 0.63379
class #1 : 0.83032
class #2 : 0.01883
class #3 : 0.73014
class #4 : 0.66084
class #5 : 0.54096
mean iou score 0.5691479156965283


Epoch: 2
Loss: 0.5263281466380579
resize...
class #0 : 0.67732
class #1 : 0.84422
class #2 : 0.16176
class #3 : 0.75009
class #4 : 0.67887
class #5 : 0.52544
mean iou score 0.6062845492449496


Epoch: 3
Loss: 0.47683974751341773
resize...
class #0 : 0.69730
class #1 : 0.85639
class #2 : 0.19858
class #3 : 0.78785
class #4 : 0.69338
class #5 : 0.61585
mean iou score 0.6415586947006878


Epoch: 4
Loss: 0.4370490254337428
resize...
class #0 : 0.69464
class #1 : 0.83327
class #2 : 0.16094
class #3 : 0.71099
class #4 : 0.69960
class #5 : 0.50913
mean iou score 0.6014310893549384


Epoch: 5
Loss: 0.4